In [ ]:
import requests
import os
import json
from datetime import datetime, timedelta

In [ ]:
import pandas as pd

`https://api.nasa.gov/neo/rest/v1/feed?start_date=START_DATE&end_date=END_DATE&api_key=API_KEY`

In [ ]:
req = requests.get("https://api.nasa.gov/neo/rest/v1/feed",
                   params={'api_key': os.environ['NASA_API_KEY'],
                           'start_date': datetime.now().strftime("%Y-%m-%d"),
                           'end_date': (datetime.now()+timedelta(days=6)).strftime("%Y-%m-%d")})

In [ ]:
def get_asteroid_data(asteroid: dict):
    link = asteroid['nasa_jpl_url']
    name = asteroid['name']
    avg_width = (asteroid['estimated_diameter']['meters']['estimated_diameter_min'] \
                 + asteroid['estimated_diameter']['meters']['estimated_diameter_max'])/2
    velocity_km_s = float(asteroid['close_approach_data'][0]['relative_velocity']['kilometers_per_second'])
    approach_date = datetime.fromtimestamp(asteroid['close_approach_data'][0]['epoch_date_close_approach']/1000)
    potentially_hazardous = int(asteroid['is_potentially_hazardous_asteroid'])
    miss_distance_km = float(asteroid['close_approach_data'][0]['miss_distance']['kilometers'])
    return dict(link=link,
                name=name,
                width_m=avg_width,
               velocity_km_s=velocity_km_s,
               approach_date=approach_date,
               potentially_hazardous=potentially_hazardous,
               miss_distance_km=miss_distance_km)

In [ ]:
def get_request_metadata(request):
    if req.status_code != 200:
        return {'error': True,
                'message': req.text}
    req_json = req.json()
    day_neo_li_dict = req_json['near_earth_objects']
    return {day: len(neo_li) for day, neo_li in day_neo_li_dict.items()}

In [ ]:
get_request_metadata(req)

In [ ]:
rock_df = pd.DataFrame(list(map(get_asteroid_data,[rock for day in req.json()['near_earth_objects'].values() for rock in day])))

In [ ]:
from bokeh.io import output_notebook, output_file

In [ ]:
output_file('plot.html')

In [ ]:
from bokeh.io import show
from bokeh.plotting import show, figure
from bokeh.models import ColumnDataSource, LinearColorMapper
from bokeh.transform import transform
from bokeh.palettes import Viridis256, Inferno256

In [ ]:
selection = rock_df[(rock_df.approach_date >= datetime.now()) &
        (rock_df.approach_date < datetime.now() + timedelta(hours=24))].sort_values('approach_date')

In [ ]:
rock_df

In [ ]:
lcm = LinearColorMapper(palette = Inferno256, low=selection.velocity_km_s.min(), high=selection.velocity_km_s.max())


In [ ]:
import json

In [ ]:
p = figure(x_axis_type='datetime', toolbar_location=None)
cds = ColumnDataSource(data=selection.assign(width_m=selection.width_m/10))
p.scatter(x='approach_date', y='miss_distance_km', size='width_m', 
          fill_color=transform('velocity_km_s', lcm),
          line_color='black',
          source=cds)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.outline_line_color = None
# p.background_fill_color = '#fffff8'

# p.axis.background_fill_color =  '#fffff8'
show(p)

In [ ]:
datasets = {'potentially_hazardous': [],
            'safe': []}
for i,row in rock_df.iterrows():
    if row['potentially_hazardous']:
        datasets['potentially_hazardous'].append(dict(x=row['approach_date'].timestamp(),
                                                      y=row['miss_distance_km'],
                                                      width=row['width_m'],
                                                      name=row['name'].strip('()') if (row['name'][0] == row['name'][-1]) and row['name'][0] in '()' else row['name'],
                                                      speed=row['velocity_km_s'])
                                                )
    else:
        datasets['safe'].append(dict(x=row['approach_date'].timestamp(),
                                     y=row['miss_distance_km'],
                                     width=row['width_m'],
                                     name=row['name'].strip('()') if (row['name'][0] == row['name'][-1]) and row['name'][0] in '()' else row['name'],
                                    speed=row['velocity_km_s'])
                               )


In [ ]:
data = dict(datasets=[dict(label=k,
                           data=datasets[k]) for k in datasets])

In [ ]:
rock_df.approach_date.max()

In [ ]:
json.dumps(data)

---

In [1]:
import classes
from importlib import reload

In [2]:
import asyncio

In [56]:
classes = reload(classes)

In [57]:
asteroids = classes.AsteroidAstronomer(n_days_from_current=6)

In [58]:
d = await asteroids._query_neo_data()

In [59]:
d

{'links': {'next': 'http://www.neowsapp.com/rest/v1/feed?start_date=2021-01-29&end_date=2021-02-04&detailed=false&api_key=7ge9BC2WU2abf5Pmi9wkqm9iS70iBeeUFPyeHQ7q',
  'prev': 'http://www.neowsapp.com/rest/v1/feed?start_date=2021-01-17&end_date=2021-01-23&detailed=false&api_key=7ge9BC2WU2abf5Pmi9wkqm9iS70iBeeUFPyeHQ7q',
  'self': 'http://www.neowsapp.com/rest/v1/feed?start_date=2021-01-23&end_date=2021-01-29&detailed=false&api_key=7ge9BC2WU2abf5Pmi9wkqm9iS70iBeeUFPyeHQ7q'},
 'element_count': 65,
 'near_earth_objects': {'2021-01-24': [{'links': {'self': 'http://www.neowsapp.com/rest/v1/neo/3313180?api_key=7ge9BC2WU2abf5Pmi9wkqm9iS70iBeeUFPyeHQ7q'},
    'id': '3313180',
    'neo_reference_id': '3313180',
    'name': '(2006 BC)',
    'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3313180',
    'absolute_magnitude_h': 19.1,
    'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.4023045798,
      'estimated_diameter_max': 0.8995803882},
     'meters': {'estimated_diame